## Webscraping

Use BeautifulSoup to get quotes, authors, and tags from [Quotes to Read](http://quotes.toscrape.com/)

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

1. Write a function to get all the authors from the [homepage](http://quotes.toscrape.com/)

In [5]:
def authors(url):
    '''
    input: url
    
    return: a dictionary of of authors and their urls
            {'author_1':'url_of_author_1', 'author_2':'url_of_author_2' ...}
    '''
    pass

In [30]:
# answer

def authors(url):
    # Make a get request to retrieve the page
    html_page = requests.get(url) 
    # Pass the page contents to beautiful soup for parsing
    soup = BeautifulSoup(html_page.content, 'html.parser')
    authors = soup.find_all('small')
    author_dictionary = {}
    for author in authors:
        author_dictionary[author.text] = author.find_next_siblings()[0].get('href')
    return author_dictionary

In [7]:
# test the function
authors('http://quotes.toscrape.com/')

{'Albert Einstein': '/author/Albert-Einstein',
 'J.K. Rowling': '/author/J-K-Rowling',
 'Jane Austen': '/author/Jane-Austen',
 'Marilyn Monroe': '/author/Marilyn-Monroe',
 'André Gide': '/author/Andre-Gide',
 'Thomas A. Edison': '/author/Thomas-A-Edison',
 'Eleanor Roosevelt': '/author/Eleanor-Roosevelt',
 'Steve Martin': '/author/Steve-Martin'}

In [8]:
# test the function
authors('http://quotes.toscrape.com/page/3')

{'Pablo Neruda': '/author/Pablo-Neruda',
 'Ralph Waldo Emerson': '/author/Ralph-Waldo-Emerson',
 'Mother Teresa': '/author/Mother-Teresa',
 'Garrison Keillor': '/author/Garrison-Keillor',
 'Jim Henson': '/author/Jim-Henson',
 'Dr. Seuss': '/author/Dr-Seuss',
 'Albert Einstein': '/author/Albert-Einstein',
 'J.K. Rowling': '/author/J-K-Rowling',
 'Bob Marley': '/author/Bob-Marley'}

2. Write a function to get the tags of each quote on a page

#### Might include the instructions to use the class tag... there are a couple ways to get this

In [110]:
def tags(url):
    '''
    input: url
    
    return: a list of dictionaries of of tags and their urls
            [{'quote_tags':[list_of_quote_1_tags]}, {'quote_tags':[list_of_quote_1_tags]}]
            
    '''
    pass

In [23]:
# answer
# this answer uses the class below, you can also use the tags themselves
# soup.find_all(class_ = 'keywords')

def tags(url):
    # Make a get request to retrieve the page
    html_page = requests.get(url) 
    # Pass the page contents to beautiful soup for parsing
    soup = BeautifulSoup(html_page.content, 'html.parser')
    
    list_of_tags = []
    for tag in soup.find_all(class_ = 'keywords'):
        quote_tags = {}
        quote_tags['quote_tags'] = tag['content'].split(',')
        list_of_tags.append(quote_tags)
    return list_of_tags

In [26]:
tags('http://quotes.toscrape.com/')

[{'quote_tags': ['change', 'deep-thoughts', 'thinking', 'world']},
 {'quote_tags': ['abilities', 'choices']},
 {'quote_tags': ['inspirational', 'life', 'live', 'miracle', 'miracles']},
 {'quote_tags': ['aliteracy', 'books', 'classic', 'humor']},
 {'quote_tags': ['be-yourself', 'inspirational']},
 {'quote_tags': ['adulthood', 'success', 'value']},
 {'quote_tags': ['life', 'love']},
 {'quote_tags': ['edison', 'failure', 'inspirational', 'paraphrased']},
 {'quote_tags': ['misattributed-eleanor-roosevelt']},
 {'quote_tags': ['humor', 'obvious', 'simile']}]

3. Write a function to get all of the quotes on a set number of pages

In [ ]:
def get_some_quotes(url, number_of_pages=None):
    '''
    input: url, number of pages to scrap (just scrape the home page if no argument is passed in)
    
    return: a list of dictionaries of quotes with their attributes
            [{'quote':'quote_1_text', 'author':'url_of_author_1', 'quote_tags':[list_of_quote_1_tags]}, 
            {'quote':'quote_2_text', 'author':'url_of_author_2', 'quote_tags':[list_of_quote_2_tags]}, ...]
    '''
    pass

In [ ]:
# answer

def get_some_quotes(url, number_of_pages=None):
    # Make a get request to retrieve the page
    html_page = requests.get(url) 
    # Pass the page contents to beautiful soup for parsing
    soup = BeautifulSoup(html_page.content, 'html.parser')
    
    tags(url)

4. From the dictionary of author get all the author's birthday and birthplace

In [10]:
# maybe use the author function to get the url for the author's page
def from_where():
    pass



`get_some_quotes` is what we will import into the mongodb


## NoSQL 

In [ ]:
import pymongo

1. Create a mongo database

2. Add the quotes from `get_some_quotes` or use the json file 

3. Update the quote...

4. delete the first quote from the database